In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

class LeNet:
    @staticmethod
    def build(input_shape=(32, 32, 1), activation='sigmoid'):
        model = Sequential()
        model.add(Conv2D(6, (5,5), input_shape=input_shape, 
                         activation=activation, 
                         kernel_initializer='random_uniform'))
        model.add(AveragePooling2D(pool_size=(2,2)))

        model.add(Conv2D(16, (5,5), 
                         activation=activation, 
                         kernel_initializer='random_uniform'))
        model.add(AveragePooling2D(pool_size=(2,2)))

        model.add(Flatten())
        model.add(Dense(120, activation=activation))
        model.add(Dense(84, activation=activation))
        model.add(Dense(10, activation='softmax'))

        return model

In [2]:
model = LeNet.build(input_shape=(28, 28, 1), activation="relu")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 12, 12, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 4, 4, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 120)               3

In [3]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd", 
              metrics=['accuracy'])

In [4]:
from tensorflow.keras.datasets import mnist

(train_X, train_y), (test_X, test_y) = mnist.load_data()

train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

train_X = train_X.astype('float32') / 255
test_X = test_X.astype('float32') / 255
print('train_X shape:', train_X.shape)
print('test_X shape:', test_X.shape)

train_X shape: (60000, 28, 28, 1)
test_X shape: (10000, 28, 28, 1)


In [5]:
hist = model.fit(train_X, train_y,
                 validation_data=(test_X, test_y),
                 batch_size=200, epochs=20)

Epoch 1/20
300/300 [==============================] - 2s 2ms/step - loss: 2.2893 - accuracy: 0.3203 - val_loss: 2.2653 - val_accuracy: 0.3826
Epoch 2/20
300/300 [==============================] - 1s 2ms/step - loss: 2.0393 - accuracy: 0.4874 - val_loss: 1.1394 - val_accuracy: 0.7248
Epoch 3/20
300/300 [==============================] - 1s 2ms/step - loss: 0.6398 - accuracy: 0.8145 - val_loss: 0.4325 - val_accuracy: 0.8749
Epoch 4/20
300/300 [==============================] - 1s 2ms/step - loss: 0.4131 - accuracy: 0.8745 - val_loss: 0.3535 - val_accuracy: 0.8923
Epoch 5/20
300/300 [==============================] - 1s 2ms/step - loss: 0.3397 - accuracy: 0.8966 - val_loss: 0.2900 - val_accuracy: 0.9105
Epoch 6/20
300/300 [==============================] - 1s 2ms/step - loss: 0.2923 - accuracy: 0.9120 - val_loss: 0.2605 - val_accuracy: 0.9195
Epoch 7/20
300/300 [==============================] - 1s 2ms/step - loss: 0.2619 - accuracy: 0.9201 - val_loss: 0.2689 - val_accuracy: 0.9168
Epoch 

In [6]:
score = model.evaluate(test_X, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.11124993860721588
Test accuracy: 0.9653000235557556


In [7]:
(loss, accuracy) = model.evaluate(test_X, test_y, batch_size=128)
print("Accuracy: {:.2f}%".format(accuracy * 100))

79/79 [==============================] - 0s 1ms/step - loss: 0.1112 - accuracy: 0.9653
Accuracy: 96.53%


In [8]:
import numpy as np
import cv2

samples = np.random.choice(np.arange(0,len(test_y)),size=(10,))

for i in samples:
    probs = model.predict(test_X[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    image = (test_X[i] * 255).astype("uint8")
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image = cv2.resize(image, (96, 96), 
                       interpolation=cv2.INTER_LINEAR)
 
    cv2.putText(image, str(prediction[0]), (5, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
    print("Predicted: {}, Actual: {}".format(prediction[0], test_y[i]))
    cv2.imshow("Digit", image)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Predicted: 7, Actual: 7
Predicted: 8, Actual: 8
Predicted: 6, Actual: 6
Predicted: 3, Actual: 3
Predicted: 3, Actual: 3
Predicted: 5, Actual: 5
Predicted: 8, Actual: 8
Predicted: 3, Actual: 3
Predicted: 1, Actual: 1
Predicted: 2, Actual: 2
